In [1]:
from Memory import ReplayMemory
from SACV2 import SoftActorCritic
# import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import argparse
import sys
sys.path.append("..")
from behavioral_sim.custom_envs import BehavSimEnv
from behavioral_sim.custom_envs import HourlySimEnv
import IPython
from keras.models import Sequential
from keras.models import model_from_json
from keras.layers import Dense
from keras.layers import LSTM
import tensorflow as tf
import pickle



Using TensorFlow backend.
/Users/lucas.spangher/Documents/Berkeley Files/Research/Deferable Loads/transactive_control/baselines-RAISE/baselines/baselines_env/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/lucas.spangher/Documents/Berkeley Files/Research/Deferable Loads/transactive_control/baselines-RAISE/baselines/baselines_env/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/lucas.spangher/Documents/Berkeley Files/Research/Deferable Loads/transactive_control/baselines-RAISE/baselines/baselines_env/lib/python3.6

In [2]:
"""
My rough code to train and experiment with SACV2, I pretty much just call the train function below


"""

replay_size = 10000


total_numsteps = 60
start_steps = 30
batch_size = 1
action_star = None

## Helper functions

In [3]:

def moving_average(a, n=3) :
    ret = np.cumsum(a, dtype=float)
    ret[n:] = ret[n:] - ret[:-n]
    return ret[n - 1:] / n

def series_to_supervised(data,
                         n_in,
                         target_features,
                         col_names=None,
                         n_out=1,
                         dropnan=True,
                         initial_excluded_timesteps=0,
                         removed_features=[],
                         only_current_timestep_features=[]):
    """Takes time series data and converts it into a supervised learning
    problem framework.

        Parameters:
            - data (pd.Dataframe) -- the time series data to be converted.
            - n_in (int) -- Number of time steps to use as lag for the feature 
                matrix
            - col_names (List[str]) -- list of strings to use as column names,
                that get converted into features for each time lag
            - target_features (List[str]) -- List of features that will be used
                as dependent variables in the target matrix.
            - n_out (int) -- Number of time steps to use as lag for the target
                matrix
            - dropnan (bool) -- Whether to drop nan values
            - initial_excluded_timesteps (int) -- The number of input timesteps to 
                ignore before starting the time lag.
            - removed_features (List[str]) -- List of features that should be removed
                from the dataframe. 
            - only_current_timestep_features (List[str]) -- Features that should
            only be included in the current timestep, not any before (e.g.) to avoid
            unintended dependencies
            
            - [Planned] exclude_current_day (bool) -- Whether to include values 
                from the current day. If this parameter is false, then the time lag 
                will always start with the day preceding the current time 
                step.
                
        Outputs:
            - (X, y): (Feature matrix, target matrix)
    """
    
    if col_names is None:
        col_names = data.columns
        
    n_vars = 1 if type(data) is list else data.shape[1]
    df = pd.DataFrame(data).drop(removed_features, axis=1)
    col_names = [x for x in col_names if x not in removed_features]
    cols, names = list(), list()      
        
    only_prev_time_features = df.drop(only_current_timestep_features, axis=1)
    only_prev_column_names = [x for x in col_names if x in only_prev_time_features.columns]

    # (t-n, ... t-1) --> i.e. steps into the past
    for i in range(n_in + initial_excluded_timesteps, initial_excluded_timesteps, -1):
        cols.append(only_prev_time_features.shift(i))
        names += [('%s(t-%d)' % (col, i)) for col in only_prev_column_names]

    # (t, t+1, ... t+n) --> i.e. steps into the future
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('%s(t)' % (col)) for col in col_names]
        else:
            names += [('%s(t + %d)' % (col, i)) for col in col_names]

    # concat
    agg = pd.concat(cols, axis=1)
#     agg.columns = names

    # dropnan
    if dropnan:
        agg.dropna(inplace=True)
    
    if target_features: 
        Y_vals_cols = ([('%s(t)' % (col)) for col in target_features] + 
            ['%s(t + %d)' % (col, i) for col in target_features for i in range(1, n_out)])

        Y_vals = agg[Y_vals_cols]
        X_vals = agg.drop(Y_vals_cols, axis=1)

        agg.columns = names
              
        return X_vals, Y_vals # X_vals.values, Y_vals.values
    
    else:
        return agg, _



In [ ]:
def planning_prediction(action, day_of_week, loaded_model):

    ## load the minMaxScalers
    with open ("scaler_X.pickle", "rb") as input_file:
        scaler_X = pickle.load(input_file) 
    with open ("scaler_y.pickle", "rb") as input_file:
        scaler_y = pickle.load(input_file) 

    ## prepare the data

    d_X = pd.DataFrame(data = { "action"  : action, "dow" : day_of_week } )
    scaled_X = scaler_X.transform(d_X)
    sxr = scaled_X.reshape((scaled_X.shape[0], 1, scaled_X.shape[1])) 

    # supervised_X, _ = series_to_supervised(data = scaled_X, 
    #                         target_features = None, # target_features = ['Energy'], 
    #                         only_current_timestep_features=[],
    #                         initial_excluded_timesteps=10,
    #                         col_names = ["Point", "Day of Week"], # df.columns.tolist(), 
    #                         n_in = 40,
    #                         n_out = 1)

    ## load the trained nn
    
    
    # other fields: (does the planning model train for tomorrow's price sig and stuff?)
                   # (yes, so it should really have the same input as the RL agent) (? ask andreea) 
        #  time lag over 20*10 timesteps 
        #  out of office (?) 
        #  vicarious learning
        #  survey responses 
        #  weather 

        # ------------
        # t = today
        # 1 step in env: (state: [10-d energy t-1, 10-d grid price t]) <- Planning & RL get to train 
        # once trained, planning: (state: [10 energy t, 10 grid price t+1]) <- RL tests diff actions
            # ? check that it samples diff actions 
            # should the state space of the RL agent be augmented to potentially 20 
                # state inputs can be downweighted? (is there a mech to remember previous states?)
            
        # TODO: 
            # augment the state space to make sure that we're passing to the agent & planning model the right stuff
                # maybe keep this stuff in the csv 
            # predict a different response per person
            # train the planning model after the day is finished 

    preds = loaded_model.predict(sxr)

    inv_preds = scaler_y.inverse_transform(preds)  

    return np.squeeze(inv_preds)

## Train the planning model with more data 

In [ ]:
def train_planning_model(
    energy_today, 
    action, 
    day_of_week, 
    loaded_model, 
    filename_to_save = "GPyOpt_planning_model"
):
    
    ## load the minMaxScalers
    with open ("scaler_X.pickle", "rb") as input_file:
        scaler_X = pickle.load(input_file) 
    with open ("scaler_y.pickle", "rb") as input_file:
        scaler_y = pickle.load(input_file) 

    ## prepare the data

    d_X = pd.DataFrame(data = { "action" : action, "dow" : day_of_week } )
    scaled_X = scaler_X.transform(d_X)
    sxr = scaled_X.reshape((scaled_X.shape[0], 1, scaled_X.shape[1])) 
    
    d_y = pd.DataFrame(data = {"energy" : energy_today})
    scaled_y = scaler_y.transform(d_y)
    
    loaded_model.fit(
            scaled_X,    ## these all need to be changed if the GPyOpt evaluates differently
            scaled_y,
            epochs=100,
            batch_size=10,
            validation_split=0.0,
            verbose=0,
        )
    
    model_json = loaded_model.to_json()
    with open(filename_to_save + ".json", "w") as json_file:
        json_file.write(model_json)
    # serialize weights to HDF5
    optimal_lstm_model.save_weights(filename_to_save + ".h5")
    print("Saved model to disk")
    
    return

## load model

In [ ]:
def load_model_from_disk(filename = "GPyOpt_planning_model"):
    json_file = open(filename + ".json", 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    loaded_model = tf.keras.models.model_from_json(loaded_model_json)
    # load weights into new model
    loaded_model.load_weights(file_name + ".h5")
    print("Loaded model from disk")

    loaded_model.compile(loss="mse", optimizer="adam")
    return loaded_model

## Train function


In [8]:
def train(response_type_str, 
    extra_train, 
    planning_iterations,
    energy=True, 
    day_of_week=True, 
    planning_model = False,
    ):
    """
    Args: 
        Response_type_str = 'theshold_exp' or 'sin' or 'mixed' or 'linear'
        Extra_Train = Number of iterations to "overtrain"
        Energy: Whether or not to include previous day energy in the state
        Day_of_Week: Whether or not to include day_of_week multiplier
    
    Summary:
        This code 'simulates' a run of SACV2 training and acting over 30 days (takes a step each day)

    """
    if(response_type_str == 'threshold_exp'):
        #env = HourlySimEnv(response='t', one_day=True, energy_in_state=False)
        env2 = BehavSimEnv(response='t', one_day=False, energy_in_state=energy, yesterday_in_state=False,
                            day_of_week = day_of_week)
    elif(response_type_str == 'sin'):
        #env = HourlySimEnv(response='s',one_day=True, energy_in_state=False)
        env2 = BehavSimEnv(response='s', one_day=False, energy_in_state=energy, yesterday_in_state=False,
                            day_of_week = day_of_week)
    elif(response_type_str == 'mixed'):
        #env = HourlySimEnv(response='s',one_day=True, energy_in_state=False)
        env2 = BehavSimEnv(response='m', one_day=False, energy_in_state=energy, yesterday_in_state=False,
                            day_of_week = day_of_week)
    elif(response_type_str == 'linear'):
        #env = HourlySimEnv(response='l',one_day=True, energy_in_state=False)
        env2 = BehavSimEnv(response='l', one_day=False, energy_in_state=energy,yesterday_in_state=False,
                            day_of_week = day_of_week)
    
    else:
        raise NotImplementedError

    #rewards in environment that agents see
    rewards = []

    #optional rewards list for environment that agent doesn't see (used in one-day training -> generalization case)
    rewards2 = []

    min_combined_losses = []
    min_policy_losses = []
    min_alpha_losses = []
    num_iters_list = []
    overall_best_action = None

    memory = ReplayMemory(replay_size)

    #Sometimes use 2 environment, this is used to default to 1 env. 
    #Change if you want to use 2 environments
    env = env2

    action_star = []
    state = None


    agent = SoftActorCritic(env.observation_space, env.action_space, memory)

    # Actions 2 save and energy_usage for data_generation
    # actions_2_save = []
    # energy_usage = []

    # Flag corresp to whether first state has been initialized
    start_flag = False

    while (env.day <= 60):
        step = env.day
        day_of_week = env.day % 7 
        print("Day: " + str(step))
        if (not start_flag):
            action = env.action_space.sample()
            next_state, reward, done, info = env.step(action)
            state = np.copy(next_state)
            start_flag = True
            continue

        if env.day <= 30:
            action = env.action_space.sample()  # Sample random action
            next_state, reward, done, info = env.step(action)

            memory.push((state, action, reward, next_state, done))

            state = np.copy(next_state)
            continue
            
        else:
            if(memory.get_len() > batch_size):
                critic_1_losses = [None]
                critic_2_losses = [None]
                policy_losses = [None]
                alpha_losses = [None]
                actions = []
                for extra_step in range(extra_train):
                    print("--"*10)
                    print(" Extra Train " + str(extra_step))
                    q1_prev_loss = critic_1_losses[-1]
                    q2_prev_loss = critic_2_losses[-1]
                    policy_loss = policy_losses[-1]
                    alpha_loss = alpha_losses[-1]
                    return_update = agent.update_params(batch_size, memory_type = "replay")
                    print(return_update)
                    print("--"*10)
                    critic_1_loss = return_update[0]
                    critic_2_loss = return_update[1]
                    policy_loss = return_update[2]
                    alpha_loss = return_update[3]

                    critic_1_losses.append(critic_1_loss)
                    critic_2_losses.append(critic_2_loss)
                    policy_losses.append(policy_loss)
                    alpha_losses.append(alpha_loss)
                    # num_iters_list.append(num_iters)
                    # actions.append(agent.get_action(state))

                energy_yesterday = np.copy(next_state[:10])

                if planning_model:
                    
                    ## load model from disk now    
                    loaded_model = load_model_from_disk("GPyOpt_planning_model")
                
                    for planning_step in range(planning_iterations):
                        print("--"*10)
                        print(" planning step " + str(planning_step))
                        q1_prev_loss = critic_1_losses[-1]
                        q2_prev_loss = critic_2_losses[-1]
                        policy_loss = policy_losses[-1]
                        alpha_loss = alpha_losses[-1]

                        grid_prices_today = env.prices[env.day] # assuming that the env.day here correctly is set to the correct day
                        grid_prices_tmr = env.prices[(env.day + 1) % 365]
                        # state is defined as [previous day energy, current day prices] 
                        # I don't currently want this to change, as it should be the same to allow agent to explore

                        state = np.concatenate((energy_yesterday, grid_prices_today))
                        action = agent.get_action(state)
                     
                        planned_energy_consumption = planning_prediction(action, day_of_week, loaded_model)
                        
                        # will define next state as [energy, grid prices tomorrow]

                        next_state = np.concatenate((planned_energy_consumption, grid_prices_tmr))
                        reward = env.get_reward_planning_model(grid_prices_today, planned_energy_consumption) 
                        done = True

                        agent.planning_replay_memory.push((state, action, reward, next_state, done))

                        return_update = agent.update_params(batch_size, memory_type = "planning")

                        print(return_update)
                        print("--"*10)
                        critic_1_loss = return_update[0]
                        critic_2_loss = return_update[1]
                        policy_loss = return_update[2]
                        alpha_loss = return_update[3]

                        critic_1_losses.append(critic_1_loss)
                        critic_2_losses.append(critic_2_loss)
                        policy_losses.append(policy_loss)
                        alpha_losses.append(alpha_loss)
                        # num_iters_list.append(num_iters)
                        # actions.append(agent.get_action(state))
            
            #Finds the action corresp to the lowest combined q-loss
            # combined_q_loss = np.array(critic_1_losses[1:]) + np.array(critic_2_losses[1:])
            # min_loss = np.amin(combined_q_loss)
            # min_combined_losses.append(min_loss)
            # index_of_min = np.where(combined_q_loss == min_loss)[0][0]
            
            action = agent.get_action(state)

            # min_policy_losses.append(np.amin(np.array(policy_losses)))
            # min_alpha_losses.append(np.amin(np.array(alpha_losses)))

            next_state, reward, done, info = env.step(action)

            memory.push((state, action, reward, next_state, done))
            
            # train the planning model 
            energy_today = np.copy(next_state[:10])
            
            
            # we may want to skip the training for now, given the simulation is 
            # off weird/different data 
            
#             train_planning_model(    
#                 energy_today = energy_today, 
#                 action = action, 
#                 day_of_week = day_of_week, 
#                 loaded_model = loaded_model)
            
            state = np.copy(next_state)

            #old code for saving data samples
            # actions_2_save.append(action[0])

            if(done):
                rewards.append(reward)
                #rewards2.append(reward2)

                #Old code for datageneration
                # env_usage = env._simulate_humans(state, action)["avg"]
                # for energy_i in env_usage:
                #     energy_usage.append(energy_i)
            
            #Old code when rewards where not fixed
            #rewards = [r[0] if r is np.ndarray else r for r in rewards]
            #rewards2 = [r[0] if r is np.ndarray else r for r in rewards2]
            print("--------" * 10)
    
    # plt.figure()
    # plt.plot(rewards, label='reward')
    # plt.plot(moving_average(rewards),label='Moving Avg')
    # plt.title("Rewards (H2H " + response_type_str + '| Total = ' + str(sum(rewards)), pad = 20.0)
    # plt.legend()
    # plt.xlabel("Day Number")
    # plt.ylabel("Reward")
    # plt.savefig(response_type_str +'_energy=' + str(energy) +'_training' + '_extratrain_' + str(extra_train) + '.png')

    # # plt.figure()
    # # plt.plot(rewards2, label='true reward')
    # # plt.plot(moving_average(rewards2),label='Moving Avg')
    # # plt.title("Daily Response (Trained on One_Day " + response_type_str + '| Total = ' + str(sum(rewards2)), pad = 20.0)
    # # plt.legend()
    # # plt.xlabel("Day Number")
    # # plt.ylabel("Reward")
    # # plt.savefig(response_type_str + '_results' + '_extratrain_' + str(extra_train) + '.png')

    # plt.figure()
    # plt.plot(min_combined_losses)
    # plt.xlabel('Iteration ')
    # plt.ylabel("Combined Q1+Q2 loss")
    # plt.title("Combined Critic Loss at each hour", pad = 20.0)
    # plt.savefig(response_type_str + '_energy=' + str(energy) + '_min_q_loss' + '_extratrain_' + str(extra_train) + '.png')

    return rewards, critic_1_losses, critic_2_losses, policy_losses, alpha_losses



In [5]:
def train_curve_finder(max_iter, response_type_str):
    def train_store_rewards(response_type_str):
        sampled_days = [19,16,29,18,14,23,9,21,10,30]
        #Key = Day | Val = list for SAC Reward
        rewards_dict = {i: [] for i in range(1,max_iter,10)}
        for iteration in range(1,max_iter,10):
            #Add error bounds, just for loop then return avg, pointwise-max/min
            for i in range(5):
                curr_rewards_sac = train(response_type_str,iteration, energy=True, day_of_week=True)
                rewards_dict[iteration].append(curr_rewards_sac)
            # if(curr_total_rewards > max_reward):
            #     max_reward = curr_total_rewards
            # if(curr_total_rewards < min_reward):
            #     min_reward = curr_total_rewards
            # cum_reward += curr_total_rewards

            # rewards_list_e.append(cum_reward / 5)
            # rewards_list_e_min.append(min_reward)
            # rewards_list_e_max.append(max_reward)
        # return np.array(rewards_list_e), np.array(rewards_list_e_min), np.array(rewards_list_e_min)
        
        return rewards_dict
        
    sac_rewards_dict = train_store_rewards(response_type_str)
    for et in sac_rewards_dict.keys():
        sac_rewards_iter_et = sac_rewards_dict[et]
        print(sac_rewards_iter_et)
        print("----\n"*2)
        np.save("SACV2_" + response_type_str + "_" + str(et) + "_rewards_no_seed",sac_rewards_iter_et)


    # rewards_thresh_e, min_rewards_thresh_e, max_rewards_thresh_e = train_store_rewards('threshold_exp')
    # rewards_sin, min_rewards_sin, max_rewards_sin = train_store_rewards('sin')
    # rewards_linear, min_rewards_linear, max_rewards_linear = train_store_rewards('linear')
    
    # total_rewards_thresh_e, total_rewards_thresh_no_e = train_store_rewards('threshold_exp')
    # total_rewards_sin_e, total_rewards_sin_no_e  = train_store_rewards('sin')
    # total_rewards_linear_e, total_rewards_linear_no_e  = train_store_rewards('linear')
    #total_rewards_mixed_e, total_rewards_mixed_no_e  = train_store_rewards(response_type_str)
    
    # plt.figure()
    # plt.plot(total_rewards_mixed_e, label=response_type_str + '-dow_-w/-energy', linestyle='dashed',color='#ED217C')
    # plt.plot(total_rewards_mixed_no_e, label=response_type_str + '-dow_-w/o-energy',color='#ED217C')

    # plt.plot(rewards_thresh_e, label='avg_threshold_exp',color='#1B998B')
    # plt.plot(rewards_sin, label='avg_sin',color = '#ED217C')
    # plt.plot(rewards_linear, label='avg_linear',color = '#2D3047')


    # plt.plot(min_rewards_thresh_e, label='min_thresh_exp', linestyle='dashed',color='#1B998B')
    # plt.plot(min_rewards_sin, label='min_sin',linestyle = 'dashed',color = '#ED217C')
    # plt.plot(min_rewards_linear, label='min_linear',linestyle='dashed', color = '#2D3047')

    # plt.plot(max_rewards_thresh_e, label='max_thresh_exp', linestyle='dashed',color='#1B998B')
    # plt.plot(max_rewards_sin, label='max_sin',linestyle = 'dashed',color = '#ED217C')
    # plt.plot(max_rewards_linear, label='max_linear',linestyle='dashed', color = '#2D3047')

    # plt.plot(total_rewards_thresh_no_e, label='threshold-exp-w/o-energy', color='#1B998B')
    # plt.plot(total_rewards_sin_no_e, label='sin-w/o-energy', color = '#ED217C')
    # plt.plot(total_rewards_linear_no_e, label='linear-w/o-energy', color = '#2D3047')
    #plt.plot(moving_average(rewards),label='Moving Avg')

#     plt.legend()
#     plt.xlabel("Number of Extra Train Iterations (Factor of 20")
#     plt.ylabel("Total Reward")
#     plt.savefig('total_reward_curve_w_error.png')



# train('threshold_exp',1)
# train('sin',1)
# train('linear',1)

# train('threshold_exp',10)
# train('sin',10)
# train('linear',10)

# train('threshold_exp',50)
# train('sin',50)
# train('linear',50)

# train('threshold_exp',100)
# train('sin',100)
# train('linear',100)

# train_curve_finder(100)


In [7]:
rewards_both, critic_1_losses_both, critic_2_losses_both, policy_losses_both, alpha_losses_both = train(response_type_str = "mixed", 
         extra_train = 40, 
         energy = True,
         day_of_week = True,
         planning_model = True,
         planning_iterations = 10
         )

BehavSimEnv Initialized
20
10
Day: 0
Day: 1
Day: 2
Day: 3
Day: 4
Day: 5
Day: 6
Day: 7
Day: 8
Day: 9
Day: 10
Day: 11
Day: 12
Day: 13
Day: 14
Day: 15
Day: 16
Day: 17
Day: 18
Day: 19
Day: 20
Day: 21
Day: 22
Day: 23
Day: 24
Day: 25
Day: 26
Day: 27
Day: 28
Day: 29
Day: 30
Day: 31
--------------------
 Extra Train 40
(34683364.0, 34534568.0, 215.89706420898438, -0.0)
--------------------
--------------------
 Extra Train 40
(16438818.0, 16350577.0, 202.16371154785156, -0.062017772346735)
--------------------
--------------------
 Extra Train 40
(19999352.0, 19908194.0, 187.90110778808594, -0.11395933479070663)
--------------------
--------------------
 Extra Train 40
(10616145.0, 10595111.0, 179.09400939941406, -0.1628684103488922)
--------------------
--------------------
 Extra Train 40
(30836828.0, 30749992.0, 191.8766632080078, -0.23605185747146606)
--------------------
--------------------
 Extra Train 40
(5906036.5, 5915489.5, 181.79933166503906, -0.27248498797416687)
-----------------

(31921916.0, 32360672.0, 221.776611328125, -4.080333232879639)
--------------------
--------------------
 Extra Train 40
(50349988.0, 51171292.0, 240.4234619140625, -4.115583419799805)
--------------------
--------------------
 Extra Train 40
(38370596.0, 38827648.0, 216.82965087890625, -4.116969585418701)
--------------------
--------------------
 Extra Train 40
(6710689.0, 6952551.0, 222.91954040527344, -4.072764873504639)
--------------------
--------------------
 Extra Train 40
(25257418.0, 25629896.0, 209.05181884765625, -3.990147113800049)
--------------------
--------------------
 Extra Train 40
(8434790.0, 8709528.0, 191.953125, -3.3747222423553467)
--------------------
--------------------
 planning step 0
Loaded model from disk
(18910798.0, 19422892.0, 252.78372192382812, -4.648316383361816)
--------------------
--------------------
 planning step 1
Loaded model from disk
(18540430.0, 19053494.0, 241.36219787597656, -4.381800651550293)
--------------------
-------------------

(17933464.0, 19059568.0, 278.6094970703125, -5.518123149871826)
--------------------
--------------------
 planning step 8
Loaded model from disk
(18336578.0, 19494532.0, 301.8077392578125, -6.408750057220459)
--------------------
--------------------
 planning step 9
Loaded model from disk
(18922258.0, 20116290.0, 276.31878662109375, -5.331784725189209)
--------------------
--------------------------------------------------------------------------------
Day: 34
--------------------
 Extra Train 40
(15123149.0, 15903829.0, 289.9373779296875, -7.417239665985107)
--------------------
--------------------
 Extra Train 40
(32871448.0, 34499764.0, 306.33673095703125, -6.443312644958496)
--------------------
--------------------
 Extra Train 40
(21950140.0, 23017046.0, 283.1272277832031, -6.701587677001953)
--------------------
--------------------
 Extra Train 40
(27222750.0, 28632082.0, 326.644775390625, -7.669950485229492)
--------------------
--------------------
 Extra Train 40
(3140039

(26261422.0, 31606006.0, 685.211181640625, -11.24804401397705)
--------------------
--------------------
 Extra Train 40
(36035008.0, 40128592.0, 514.1924438476562, -10.939825057983398)
--------------------
--------------------
 Extra Train 40
(7783955.5, 10489079.0, 621.699951171875, -10.268317222595215)
--------------------
--------------------
 Extra Train 40
(3876314.0, 6226300.5, 713.7515869140625, -11.35509204864502)
--------------------
--------------------
 planning step 0
Loaded model from disk
(41230476.0, 45989324.0, 510.5998229980469, -9.045613288879395)
--------------------
--------------------
 planning step 1
Loaded model from disk
(19739560.0, 24466650.0, 694.12060546875, -11.542738914489746)
--------------------
--------------------
 planning step 2
Loaded model from disk
(15916679.0, 19904948.0, 646.9581909179688, -10.503984451293945)
--------------------
--------------------
 planning step 3
Loaded model from disk
(36277248.0, 40915776.0, 544.56689453125, -10.4389114

Loaded model from disk
(37656960.0, 47605040.0, 929.732421875, -12.889081001281738)
--------------------
--------------------------------------------------------------------------------
Day: 37
--------------------
 Extra Train 40
(25490730.0, 36445728.0, 1188.115966796875, -15.506593704223633)
--------------------
--------------------
 Extra Train 40
(6420261.5, 11345200.0, 1022.0653076171875, -14.436394691467285)
--------------------
--------------------
 Extra Train 40
(11348121.0, 18670500.0, 1098.4764404296875, -11.232694625854492)
--------------------
--------------------
 Extra Train 40
(81110816.0, 102774432.0, 1316.5887451171875, -14.426143646240234)
--------------------
--------------------
 Extra Train 40
(11501009.0, 18123492.0, 1058.795654296875, -14.904729843139648)
--------------------
--------------------
 Extra Train 40
(3376063.5, 7637771.5, 1118.864013671875, -14.9787015914917)
--------------------
--------------------
 Extra Train 40
(6213022.5, 11419182.0, 1076.996

...

Loaded model from disk
(2122225.75, 240898048.0, 17154.494140625, -65.27810668945312)
--------------------
--------------------------------------------------------------------------------
Day: 55
--------------------
 Extra Train 40
(854758.0625, 254346112.0, 15090.8466796875, -66.56061553955078)
--------------------
--------------------
 Extra Train 40
(27569196.0, 274253280.0, 11592.5078125, -67.9671401977539)
--------------------
--------------------
 Extra Train 40
(23318270.0, 293468960.0, 12405.5654296875, -67.51795196533203)
--------------------
--------------------
 Extra Train 40
(8332530.5, 273831648.0, 13931.251953125, -66.41377258300781)
--------------------
--------------------
 Extra Train 40
(30378.37890625, 161248160.0, 12806.509765625, -68.04901123046875)
--------------------
--------------------
 Extra Train 40
(2572424.5, 310081408.0, 16091.3466796875, -66.62785339355469)
--------------------
--------------------
 Extra Train 40
(2300145.5, 241936512.0, 14267.1376953

(8040.3447265625, 186686192.0, 13841.2109375, -72.18773651123047)
--------------------
--------------------
 Extra Train 40
(5135176.5, 197968176.0, 12091.126953125, -71.85954284667969)
--------------------
--------------------
 Extra Train 40
(297136.78125, 219721616.0, 14361.8447265625, -72.36700439453125)
--------------------
--------------------
 Extra Train 40
(659050.625, 206821536.0, 13858.166015625, -72.3058853149414)
--------------------
--------------------
 Extra Train 40
(15803901.0, 204939712.0, 18518.787109375, -72.46793365478516)
--------------------
--------------------
 Extra Train 40
(17570252.0, 178725440.0, 17844.056640625, -71.01956176757812)
--------------------
--------------------
 Extra Train 40
(2129025.75, 148026272.0, 13686.92578125, -72.6397476196289)
--------------------
--------------------
 Extra Train 40
(15305486.0, 206044304.0, 18488.8125, -71.1375961303711)
--------------------
--------------------
 Extra Train 40
(814696.625, 208762512.0, 13835.8330

Loaded model from disk
(11073049.0, 447635744.0, 18031.703125, -76.32981872558594)
--------------------
--------------------
 planning step 3
Loaded model from disk
(9276260.0, 302817440.0, 14652.603515625, -76.55817413330078)
--------------------
--------------------
 planning step 4
Loaded model from disk
(2985052.5, 296794816.0, 19236.37109375, -75.0635757446289)
--------------------
--------------------
 planning step 5
Loaded model from disk
(7663056.5, 185587696.0, 16647.33203125, -76.10484313964844)
--------------------
--------------------
 planning step 6
Loaded model from disk
(10037548.0, 266312592.0, 13427.2275390625, -76.38232421875)
--------------------
--------------------
 planning step 7
Loaded model from disk
(7521295.5, 187402256.0, 16687.421875, -75.94776153564453)
--------------------
--------------------
 planning step 8
Loaded model from disk
(2017370.875, 188953408.0, 15195.1875, -76.84494018554688)
--------------------
--------------------
 planning step 9
Load

(3835095.0, 430801152.0, 18744.58203125, -76.77294921875)
--------------------
--------------------
 Extra Train 40
(4076108.75, 198833792.0, 16141.83203125, -80.92286682128906)
--------------------
--------------------
 Extra Train 40
(1810060.125, 353737120.0, 17763.640625, -80.72936248779297)
--------------------
--------------------
 Extra Train 40
(1772497.0, 395234592.0, 18586.8203125, -80.6136245727539)
--------------------
--------------------
 Extra Train 40
(9591209.0, 311213024.0, 14785.81640625, -81.88215637207031)
--------------------
--------------------
 Extra Train 40
(4715480.0, 261876064.0, 14313.5693359375, -81.26316833496094)
--------------------
--------------------
 Extra Train 40
(3888579.75, 200834960.0, 16166.20703125, -81.47521209716797)
--------------------
--------------------
 planning step 0
Loaded model from disk
(10137191.0, 344588608.0, 15678.736328125, -80.4297103881836)
--------------------
--------------------
 planning step 1
Loaded model from disk


Loaded model from disk
(5321663.5, 353906336.0, 21337.349609375, -77.48040008544922)
--------------------
--------------------
 planning step 8
Loaded model from disk
(3519808.25, 381442784.0, 21708.05078125, -83.5500717163086)
--------------------
--------------------
 planning step 9
Loaded model from disk
(23512.5078125, 282508064.0, 17266.30859375, -83.74954986572266)
--------------------
--------------------------------------------------------------------------------


In [ ]:
rewards_extra_train, critic_1_losses_extra_train, critic_2_losses_extra_train, policy_losses_extra_train, alpha_losses_extra_train = train(response_type_str = "mixed", 
         extra_train = 40, 
         energy = True,
         day_of_week = True,
         planning_model = False,
         planning_iterations = 10
         )

In [ ]:
rewards_plan, critic_1_losses_plan, critic_2_losses_plan, policy_losses_plan, alpha_losses_plan = train(response_type_str = "mixed", 
         extra_train = 0, 
         energy = True,
         day_of_week = True,
         planning_model = True,
         planning_iterations = 40
         )

In [ ]:
rewards_wo, critic_1_losses_wo, critic_2_losses_wo, policy_losses_wo, alpha_losses_wo = train(response_type_str = "mixed", 
         extra_train = 0, 
         energy = True,
         day_of_week = True,
         planning_model = False,
         planning_iterations = 10
         )